## Apologies for the inconvenience: I had to clear the output on all the cells to get the file small enough to drop into the git via the website. They should run pretty quickly, though!

1. A Stochastic Lotke-Voterra model is given by the following equations for the abundance of two species, Y1 and Y2:

$$Y1 \xrightarrow{c_1} 2Y1 $$
$$Y1 + Y2 \xrightarrow{c_2} 2Y2 $$
$$Y2 \xrightarrow{c_3} \emptyset $$

a) Why is this called a predator-prey model? Explain whether species 1 or species 2 can exist in the absence of the other species. What is the corresponding ODE model? 
 

a) In this model, species 1 serves both to reproduce itself and to support the reproduction of species 2. Species 2 requires species 1 to reproduce, but species 1 does not require species 2; morever, a member of species 1 is consumed during the reproduction of species 2. So the 'prey' species, species 1, can increase in the absence of 'predator' species 2 but the converse is not true. 

$$Y1' = c_1Y1 - c_2(Y1*Y2) $$ $$Y2' = c_2(Y1*Y2) - c_3Y2 $$

b) Use the Gillespie algorithm to simulate trajectories from the model. Use $c_1 = 1$, $c_2 = 0.005$, and $c_3 = 0.6$. To which parameter is species 1 extinction more sensitive? species 2?

In [ ]:
# This uses original direct ssa
import numpy as np
import matplotlib.pyplot as plt

# 1) Initialize





# conduct one run
def runLV(args):
    
    y1,y2,C1,C2,C3,numSteps = args
    Y1 = np.zeros(numSteps); Y2 = np.zeros(numSteps)
    timeline = np.zeros(numSteps); t = 0
    yup = True
    
    # run reactions
    for i in range(numSteps):
        # record
        Y1[i] = y1; Y2[i] = y2; timeline[i] = t
        
        # normalize rates
        c1 = C1*y1; c2 = C2*y1*y2; c3 = C3*y2
        total = c1 + c2 + c3
        
        # 'Yup' tells us whether to attempt to divide by total. 
        # It total is zero, it just finishes running our time out.
        if total == 0: yup = False; total = 10
        else:
            c1 = c1/total; c2 = c2/total; c3 = c3/total
        
        # decide which reaction will occur and change y values accordingly
        if(yup):
            check = np.random.uniform()
            if check < c1:
                # reaction c1 occurs
                y1  += 1
            else:
                if check > 1-c3:
                    # reaction c3 occurs
                    y2 = y2 -1

                else:
                    #reaction c2 occurs
                    y1 = y1 -1
                    y2 = y2 +1

            # check to make sure nothing's nonnegative
            y1 = max(y1,0); y2 = max(y2,0)
        
        # update time: 
        t += np.log(1/np.random.uniform())/total
        
    return Y1, Y2,timeline

In [ ]:
# Graphing function for L-V
import seaborn as sns
import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def graphLV(args, numCol):
    

    subTitles = ["Run " + str(i + 1) for i in range(numCol*2)]
    subTitles[numCol] = " "
    subTitles[numCol+1:numCol*2] = ["Run " + str(int(i)) for i in np.linspace(numCol+1,numCol*2,numCol-1)]
    fig = make_subplots(rows=2, cols=numCol,subplot_titles = subTitles)

    Y1, Y2,timeline = runLV(args)
    fig.append_trace(go.Scatter(x=timeline,y=Y1,mode = 'lines', 
                                name = "prey",line = dict(color = 'royalblue')), 
                     row=1, col=1)
    fig.append_trace(go.Scatter(x=timeline,y=Y2,mode = 'lines',
                                name = "predator",line = dict(color = 'firebrick')),
                     row = 1,col = 1)
    
    for i in range(2):
        for j in range(numCol-1):
            Y1, Y2,timeline = runLV(args)
            fig.append_trace(go.Scatter(x=timeline,y=Y1,mode = 'lines',
                                        showlegend=False,line = dict(color = 'royalblue')), 
                             row=i+1, col=j+2)
            fig.append_trace(go.Scatter(x=timeline,y=Y2,mode = 'lines',
                                        showlegend = False,line = dict(color = 'firebrick')),
                             row=i+1, col=j+2)
        
 
    fig.update_layout(title='Predator-Prey for prey: ' + str(args[0]) + "; predator: " + str(args[1]) + ";  " + "c1 = " + str(args[2]) + ", c2 = " + str(args[3]) + ", c3 = " + str(args[4]),
                   xaxis_title='time',
                   yaxis_title='population',
                     height = 500,
                      legend=dict(
                        yanchor="bottom",
                          y = .05,
                        xanchor="left",
                          x = .05
                        ))
    fig.show()

To address the above questions, fixed prey (species 1) initial population and predator (species 2) initial population, then varied the other parameters by doubling their original value and ran several trials with each parameter set. 

Doubling C1 appears to result in relatively large increases in predator population, and doubling C3 appears to often result in predator suppression and release of the prey species.

In [ ]:
# Original
y1 = 90; y2 = 90
C1 = 1; C2 = .005; C3 = .6
numSteps = 30000
args = (y1,y2,C1,C2,C3,numSteps)
numCol = 5

graphLV(args,numCol)
# Changing C1
C1 = 2; C2 = .005; C3 = .6
numSteps = 20000
args = (y1,y2,C1,C2,C3,numSteps)
graphLV(args,numCol)

# Changing C2
C1 = 1; C2 = .01; C3 = .6
args = (y1,y2,C1,C2,C3,numSteps)
numSteps = 30000
graphLV(args,numCol)

# Changing C3
C1 = 1; C2 = .005; C3 = 1.2
numSteps = 100000
args = (y1,y2,C1,C2,C3,numSteps)
graphLV(args,numCol)

2) Consider the reactions
$$A \xrightarrow{k} X \xrightarrow{\alpha_1} \emptyset$$
$$B \xrightarrow{k}Y\xrightarrow{\alpha_2} \emptyset $$
$$X + Y \xrightarrow{k_a} C $$
which has deterministic equations
$$[X]' = k - \alpha_1[X]-k_a[X][Y]$$
$$[Y]' = k - \alpha_2[Y] - k_a[X][Y]$$

a) Find the fixed points of the deterministic system. Show that for the values
{$k = 10$, $\alpha_1 = 10^{-6}$, $\alpha_2 = 10^{-5}$, $k_a=10^{-5}$} and {$k=10^3$, $\alpha_1 = 10^{-4}$, $\alpha_2 = 10^{-3}$, $k_a = 10^{-3}$} the fixed points are the same.


At the fixed points the derivatives of the concentrations should be zero. Setting both derivatives to zero and filling in the rate constants gives $$ X = \frac{10^7}{1+10Y}, Y = \frac{10^6}{1+X} $$ and $$X = \frac{\alpha_2}{\alpha_1}Y = 10Y $$ as possible fixed points for both sets of parameters.

b) Run the Gillespie algorithm and show that the behavior is very different in the two cases. Compute the stationary distribution in the two cases.

In [ ]:
def runChem(args,numSteps):
    
    # initialize
    x,y,K,A1,A2,KA = args
    X = np.zeros(numSteps); Y = np.zeros(numSteps)    
    timeline = np.zeros(numSteps); t = 0
    
    # run reactions
    for i in range(numSteps):
        # record
        X[i] = x; Y[i] = y; timeline[i] = t
        
        # normalize rates
        a1 = A1*x; ka = KA*x*y; a2 = A2*y
        total = K + a1 + a2 + ka
        k = K/total; a1 = a1/total; a2 = a2/total; ka = ka/total
        
        # decide which reaction will occur and change y values accordingly
        check = np.random.uniform()
        if check < k:
            # reaction k occurs
            x += 1; y += 1
        else:
            if check > 1-ka:
                # reaction ka occurs
                x -= 1; y -=1
            else:
                if check > 1 - (a2 + ka):
                    #reaction a2 occurs
                    y -= 1
                else:
                    #reaction a1 occurs
                    x -= 1
                
        # check to make sure everything's nonnegative
        x = max(x,0); y = max(y,0)
        
        # update time: 
        t += np.log(1/np.random.uniform())/total
        
    return X, Y,timeline

In [ ]:
import numpy as np
import seaborn as sns
import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import math 

def graphChem(arg1, arg2):
    numSteps = 10000; 


    
    # create graphs
    fig = make_subplots(rows=1, cols = 2, 
                        subplot_titles = ['Parameter set 1 (k = 10, etc.)',
                                          'Parameter set 2 (k = 1000,etc.)'])
    color_list = ['aqua','blueviolet','orange','crimson','forestgreen','fuchsia','lightgreen','tomato','darkslateblue']

    # get data for first set of arguements
    X,Y,timeline = runChem(arg1,numSteps)
    xavg = np.mean(X[math.ceil(numSteps/2):-1]);yavg = np.mean(Y[math.ceil(numSteps/2):-1]);
    xlabel =  r"$X_{avg} = " + str(round(xavg,2))+" $" ; ylabel = r"$Y_{avg} = "+ str(round(yavg,2))+"$" 
    fig.append_trace(go.Scatter(x=timeline,y=X,mode = 'lines', 
                                name =xlabel,line = dict(color = color_list[0])), row=1, col=1)
    fig.append_trace(go.Scatter(x=timeline,y=Y,mode = 'lines',
                                name = ylabel,line = dict(color = color_list[1])),row = 1,col = 1)
    
    # get data for second set of arguements
    X,Y,timeline = runChem(arg2,numSteps)
    xavg = np.mean(X[math.ceil(numSteps/2):-1]);yavg = np.mean(Y[math.ceil(numSteps/2):-1]);
    xlabel =  r"$X_{avg} = " + str(round(xavg,2))+" $" ; ylabel = r"$Y_{avg} = "+ str(round(yavg,2))+"$" 
    fig.append_trace(go.Scatter(x=timeline,y=X,mode = 'lines', 
                                name = xlabel,line = dict(color = color_list[2])), row=1, col=2)
    fig.append_trace(go.Scatter(x=timeline,y=Y,mode = 'lines',
                                name = ylabel,line = dict(color = color_list[3])),row = 1,col = 2)
    

   # titleStr = "{ 𝑘=10 ,  𝛼1=10−6 ,  𝛼2=10−5 ,  𝑘𝑎=10−5 } and { 𝑘=103 ,  𝛼1=10−4 ,  𝛼2=10−3 ,  𝑘𝑎=10−3 }"
    fig.update_layout(title=r"$X_0 = " + str(arg1[0]) + "; " + r"Y_0 = " + str(arg1[1])+"$",
                   xaxis_title='time',
                   yaxis_title='concentration',
                     height = 300,
                     legend=dict(
                        orientation="h",
                        yanchor="bottom",
                        y=-0.5,
                        xanchor="right",
                        x=1
                        ))
    fig.show()

In [ ]:
# x,y,K,A1,A2,KA = args


x = 1000; y =90
arg1 = (x,y,10,10**(-6),10**(-5),10**(-5))
arg2 = (x,y,10**(3),10**(-4),10**(-3),10**(-3))
graphChem(arg1,arg2)


x = 3000; y = 2000
arg1 = (x,y,10,10**(-6),10**(-5),10**(-5))
arg2 = (x,y,10**(3),10**(-4),10**(-3),10**(-3))
graphChem(arg1,arg2)


x = 30; y = 200
arg1 = (x,y,10,10**(-6),10**(-5),10**(-5))
arg2 = (x,y,10**(3),10**(-4),10**(-3),10**(-3))
graphChem(arg1,arg2)

For any initial set of X and Y values, the two parameter sets appear to settle into the same stationary X,Y distribution; however, parameter set 2 gets there much faster. This is probably due to the increased growth rate $k$ of both species in parameter set 2, which quickly outstrips the increase of the decay constants.

3. autoregulating gene: The deterministic set of equations describing mRNA ($r$) and protein ($p$) expression are given by
$$r' = k_l + \phi(p) - \gamma_rr ,$$
$$p' = rk_p - \gamma_pp $$
where $\phi(p)$ is a function describing the rate of mRNA transcription's dependance on protein concentration $p$.

a) Write down the transition matrix for the markov process describing this system

For (using the description from lecture 8) $$T =  \begin{bmatrix}
k_l \\ k_p \\ \gamma_r \\ \gamma_p \\ \phi(p) \end{bmatrix}
 P = \begin{bmatrix} r \\ p \end{bmatrix}, $$ we have the transition matrix
$$A = \begin{bmatrix} 1 & 0 \\ 0 & 1 \\ -1 & 0 \\ 0 & -1 \\ 1 & 0 \end{bmatrix} $$ 

b) Consider the case of positive autoregulation:
$$\phi(p) = \frac{k_0(\frac{p}{K})^n}{1+(\frac{p}{K})^n},$$
Setting $k_l = 0$, $\gamma_p = \gamma_r = k_p = k_0 = 1$ and $K = 0.5$, determine the number of fixed points for $ n = 1$ and $n = 10$. Determine the stability of the fixed points.


With these constants, our system becomes
$$r' = \frac{2p}{1+2p}-r, $$
$$p' = r-p. $$

Because $r$'s growth term $\phi(p) < 1$ for any nonnegative value of $p$, this system must go to extinction, as in the following graph.

In [ ]:
# Functions for both positive and negative autoregulation. 
# Will change which is which depending on the version of 
# function phiP passed as argument.

def runGene(args,numSteps,phiP):
    
    # initialize
    r,p,KL,KP,GR,GP = args
    R = np.zeros(numSteps); P = np.zeros(numSteps)    
    timeline = np.zeros(numSteps); t = 0
    yup = True
    
    
    # run reactions
    for i in range(numSteps):
        # record
        R[i] = r; P[i] = p; timeline[i] = t
        
        # normalize rates
        phi = phiP(p); gr = GR*r; kp = KP*r; gp = GP*p
        total = KL + phi + gr + kp + gp
        if total == 0: yup = False; total = 10000
        else:
            kl = KL/total; phi = phi/total; gr = gr/total; kp = kp/total; gp = gp/total
        
        # decide which reaction will occur and change r,p values accordingly
        # Could have combined check for k_l and \phi_p, but chose not to for clarity
        if(yup):
            check = np.random.uniform()
            if check < kl:
                # reaction k_l occurs
                r += 1
            else:
                if check > 1-gp:
                    # reaction \gamma_p occurs
                    p -= 1
                else:
                    if check < kl + phi:
                        #reaction \phi(p) occurs
                        r += 1
                    else:
                        if check < 1- (kp + gp):
                            #reaction kp occurs
                            p += 1
                        else:
                            #reaction gr occurs
                            r -= 1
                        
                
        # check to make sure everything's nonnegative
        r = max(r,0); p = max(p,0)
        
        # update time: 
        t += np.log(1/np.random.uniform())/total
        
    return R, P,timeline

In [ ]:
import matplotlib.pyplot as plt

r = 50 ; p = 40; KL = 0
KP = 1; GR = 1; GP = 1
args = (r,p,KL,KP,GR,GP)

def phiP(p,N = 10, K0 = 1, K = 0.5):
    phi = (K0*(p/K)**N)/(1+(p/K)**N)
    return phi

numSteps = 20000
R, P,timeline = runGene(args,numSteps,phiP)

plt.figure()
plt.plot(timeline,R,label = 'R')
plt.plot(timeline,P,label = 'P')
plt.legend()
plt.title("N = 10")

def phiP(p,N = 1, K0 = 1, K = 0.5):
    phi = (K0*(p/K)**N)/(1+(p/K)**N)
    return phi

numSteps = 20000
R, P,timeline = runGene(args,numSteps,phiP)

plt.figure()
plt.plot(timeline,R,label = 'R')
plt.plot(timeline,P,label = 'P')
plt.legend()
plt.title("N = 1")

c) Consider the case of negative autoregulation:
$$\phi(p) = \frac{k_0}{1+(\frac{p}{K})^n} $$

Use the Gillespie algorithm to find and plot the stationary distribution of protein for the stochastic process. Use transition rates {$k_l = 0.001$, $k_r = 0.01$, $k_p = 0.17$, $k_0 = 0.01$, and $\gamma_p=0.00028$, $\gamma_r = 0.0083$} with $n = 10$. Do this for the case of 
(i) strong regulation, $K = 100$ and 
(ii) weak regulation, $K = 10000.$
Calculate the ratio of the standard deviation to the mean of the distribution in both cases.

In [ ]:
import statistics as st
def graphGene(args,numSteps,strongPhi,weakPhi):
    #numSteps = 100; 

    # create graphs
    fig = make_subplots(rows=1, cols = 2, 
                        subplot_titles = ['Strong Regulation',
                                          'Weak Regulation'])
    color_list = ['aqua','blueviolet','orange','crimson','forestgreen','fuchsia','lightgreen','tomato','darkslateblue']

    # get data for first set of arguements
    R,P,timeline = runGene(args,numSteps,strongPhi)
    ravg = np.mean(R[math.ceil(numSteps/4):-1]);pavg = np.mean(P[math.ceil(numSteps/4):-1]);
    rstd = st.stdev(R[math.ceil(numSteps/4):-1]);pstd = st.stdev(P[math.ceil(numSteps/4):-1])
    rlabel =  r"$R_{ratio} = " + str(round(rstd/ravg,2))+" $" ; plabel = r"$P_{ratio} = "+ str(round(pstd/pavg,2))+"$" 
    fig.append_trace(go.Scatter(x=timeline,y=R,mode = 'lines', 
                                legendgroup = "strong",
                                name =rlabel,line = dict(color = color_list[0])), row=1, col=1)
    fig.append_trace(go.Scatter(x=timeline,y=P,mode = 'lines',
                                legendgroup = "strong",
                                text=[r"$r_{avg} = " + str(round(ravg,2))+"$", 
                                      r"$p_{avg} + " + str(round(pavg,2))+"$"],
                                textposition="top center",
                                name = plabel,line = dict(color = color_list[2])),row = 1,col = 1)
    fig.append_trace(go.Scatter(x=[100000,250000],y=[5,5],
                                legendgroup = "weak",mode = 'markers+text',
                                text=[r"$r_{avg} = " + str(round(ravg,2))+"$", 
                                      r"$p_{avg} = " + str(round(pavg,2))+"$"],
                                textposition="top center",
                                showlegend=False,line = dict(color = color_list[2])),row = 1,col = 1)
    
        # get data for second set of arguements
    R,P,timeline = runGene(args,numSteps,weakPhi)
    ravg = np.mean(R[math.ceil(numSteps/4):-1]);pavg = np.mean(P[math.ceil(numSteps/4):-1]);
    rstd = st.stdev(R[math.ceil(numSteps/4):-1]);pstd = st.stdev(P[math.ceil(numSteps/4):-1])
    rlabel =  r"$R_{ratio} = " + str(round(rstd/ravg,2))+" $" ; plabel = r"$P_{ratio} = "+ str(round(pstd/pavg,2))+"$" 
    
    fig.append_trace(go.Scatter(x=timeline,y=R,mode = 'lines', 
                                legendgroup = "weak",
                                name = rlabel,line = dict(color = color_list[1])), row=1, col=2)
    fig.append_trace(go.Scatter(x=timeline,y=P,mode = 'lines',
                                legendgroup = "weak",
                                text=[r"$r_{avg} = " + str(round(ravg,2))+"$", 
                                      r"$p_{avg} + " + str(round(pavg,2))+"$"],
                                textposition="top center",
                                name = plabel,line = dict(color = color_list[3])),row = 1,col = 2)
    fig.append_trace(go.Scatter(x=[100000,250000],y=[5,5],
                                legendgroup = "weak",mode = 'markers+text',
                                text=[r"$r_{avg} = " + str(round(ravg,2))+"$", 
                                      r"$p_{avg} = " + str(round(pavg,2))+"$"],
                                textposition="top center",
                                showlegend=False,line = dict(color = color_list[3])),row = 1,col = 2)
    


    fig.update_layout(title=r"$R_0 = " + str(args[0]) + "; " + r"P_0 = " + str(args[1])+"$",
                   xaxis_title='time',
                   yaxis_title='concentration',
                     height = 300,
                     legend=dict(                        
                         orientation="h",
                        yanchor="bottom",
                        y=-0.65,
                        xanchor="right",
                        x=1
                        ))
    fig.show()

In [ ]:
r = 1 ; p = 3; KL = 0.001
KP = 0.17; GR = 0.0083; GP = 0.00028
args = (r,p,KL,KP,GR,GP)

# Strong regulation
def strongPhiP(p,N = 10, K0 = 0.01, K = 100):
    phi = K0/(1+(p/K)**N)
    return phi

def weakPhiP(p,N = 10, K0 = 0.01, K = 10000):
    phi = K0/(1+(p/K)**N)
    return phi

numSteps = 10000
graphGene(args,numSteps,strongPhiP,weakPhiP)


r = 10; p = 5; args = (r,p,KL,KP,GR,GP)
graphGene(args,numSteps,strongPhiP,weakPhiP)

r = 20; p = 10; args = (r,p,KL,KP,GR,GP)
graphGene(args,numSteps,strongPhiP,weakPhiP)

r = 200; p = 100; args = (r,p,KL,KP,GR,GP)
graphGene(args,numSteps,strongPhiP,weakPhiP)

For both strong and weak regulation cases, there appears to be a stationary distribution with $r_{avg} \approx .56$ and $p_{avg}$ somewhere around 2. Averages are marked in the graphs; ratios of standard deviation to the average are given in the legend.